In [7]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY") 

In [8]:
import requests

url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}
data = {
    "model": "gpt-3.5-turbo-0613",
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"}
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    }
                },
                "required": ["location"]
            }
        }
    ]
}

response = requests.post(url, headers=headers, json=data)

In [9]:
response.json()

{'id': 'chatcmpl-7U4TfTFB4Cz2yKAl9CvwO9dQoO0tD',
 'object': 'chat.completion',
 'created': 1687401407,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n  "location": "Boston, MA"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 82, 'completion_tokens': 18, 'total_tokens': 100}}

In [10]:
response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Boston, MA"\n}'}}

In [11]:
def classify(input_string: str) -> str:
    functions = [{
        "name": "print_sentiment",
        "description": "A function that prints the given sentiment",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "string",
                    "enum": ["positive", "negative", "neutral"],
                    "description": "The sentiment."
                },
            },
            "required": ["sentiment"],
        }
    }]
    messages = [{"role": "user", "content": input_string}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call={"name": "print_sentiment"}
    )
    function_call = response.choices[0].message["function_call"]
    argument = json.loads(function_call["arguments"])
    return argument

In [12]:
response = classify("the product is bad")

In [13]:
response

{'sentiment': 'negative'}